In [ ]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt
from scipy import stats
import seaborn as sns

from itables import init_notebook_mode

init_notebook_mode(all_interactive=True)

In [ ]:
# Cargar los datos
try:
    df = pd.read_csv('../data/transacciones_retail.csv', encoding='utf-8')
except UnicodeDecodeError:
    print("except")
    df = pd.read_csv('../data/transacciones_retail.csv', encoding='ISO-8859-1')

print("Datos cargados exitosamente.")
print("Forma del dataset:", df.shape)
print("\nPrimeras 5 filas:")
print(df.head())

Por prioridad se atenderan los siguientes puntos:

1. Datos Duplicados
2. Datos Faltantes
3. Datos desbalanceados
4. Datos sesgados

## 1. Análisis de los datos los duplicados

In [ ]:
df_duplicates = df[df.duplicated()]
all_duplicates = df[df.duplicated(keep=False)]

print(f"# of rows duplicates: {len(df_duplicates)}")
print(f"All of rows duplicates: {len(all_duplicates)}")
print(f"Unique rows: {df.shape[0] - df_duplicates.shape[0]}")

### Eliminamos Duplicados

In [ ]:
df = df.drop_duplicates()

## 2. Análisis de los datos nulos

<img src="../reports/imgs/missing_values.png" alt="Datos imbalanceados">

In [ ]:
country_stats = df.groupby("Country").agg({
    'CustomerID': ['count', 'nunique'],
    'InvoiceNo': 'nunique',
    'Quantity': 'sum'
}).round(2)

country_stats.columns = ['Total_Transacciones', 'Clientes_Unicos', 'Facturas_Unicas', 'Cantidad_Total']
country_stats = country_stats.sort_values('Total_Transacciones', ascending=False)

In [ ]:
print("Top 10 países por número de transacciones:")
country_stats.head(10)

In [ ]:
print("Ultimos 10 países por número de transacciones:")
country_stats.tail(10)

In [ ]:
df.query("Country == 'Hong Kong'")

Hong kong no tiene clientes unicos ni transacciones, pero si muchos productos vendidos!!!!

In [ ]:
df.query("Country == 'Hong Kong'").shape

In [ ]:
customerId_null = df.groupby('Country').agg({
    'CustomerID': ['count', lambda x: x.isnull().sum(), 'nunique']
}).round(2)

customerId_null.columns = ['Total_Transacciones', 'CustomerID_Nulos', 'Clientes_Unicos']
customerId_null['Porcentaje_nulls'] = (customerId_null['CustomerID_Nulos'] / customerId_null['Total_Transacciones'] * 100).round(2)

# Ordenar por número de nulls descendente
customerId_null = customerId_null.sort_values('CustomerID_Nulos', ascending=False)

print("=== ANÁLISIS DE CUSTOMERID NULOS POR PAÍS ===")
customerId_null[customerId_null['CustomerID_Nulos'] > 0]

In [ ]:
country_problems = df.groupby('Country').agg({
    'CustomerID': lambda x: x.isnull().sum()
}).query('CustomerID > 0').sort_values('CustomerID', ascending=False)

print("Países con CustomerID nulos:")
for country, nulls in country_problems['CustomerID'].items():
    total = len(df[df['Country'] == country])
    percent = (nulls / total) * 100
    print(f"  {country}: {nulls} nulos de {total} transacciones ({percent:.1f}%)")

Si bien, desde el incio se observo que CustomerID, tenia varios datos nulos, se indago un poco más. Tomando en consideración las tracciones realizadas por los clientes.

Si bien, un pais a considerar es Hong Kong, con 284 registros. Sin embargo tiene los registros de CustomerID nulos. Ademas de este se observa que hay otros paises con una problematica similar. Sin embargo, se obta por eliminar aquellos registros que tienen como nulo CustomerID. Ya que el problema de negocio es el siguiente:

Construir un modelo que pueda predecir si un **cliente existente** volverá a comprar en el futuro.

In [ ]:
df = df[df['CustomerID'].notna()].copy()

In [ ]:
df.shape

# 3. Análisis Datos Desbalanceados

In [ ]:
print("=== ANÁLISIS DE DESBALANCE EN COUNTRY ===")

country_distribution = df['Country'].value_counts()
total_transactions = len(df)

print(f"Total de países: {len(country_distribution)}")
print(f"Total de transacciones: {total_transactions}")


print(f"\nTop 5 países concentran: {country_distribution.head(5).sum() / total_transactions * 100:.1f}% de las transacciones")
print(f"UK concentra: {country_distribution['United Kingdom'] / total_transactions * 100:.1f}% de las transacciones")


print("\nDistribución por país:")
for i, (country, count) in enumerate(country_distribution.head(10).items()):
    percent = (count / total_transactions) * 100
    print(f"{i+1:2d}. {country}: {count:,} ({percent:.1f}%)")

In [ ]:
# Clasificar por tiers

def classify_countries_by_volume(df):
    country_stats = df['Country'].value_counts()
    
    # Definir tiers
    tier1 = country_stats[country_stats >= 1000]      # Países principales
    tier2 = country_stats[(country_stats >= 100) & (country_stats < 1000)]  # Países medianos
    tier3 = country_stats[country_stats < 100]        # Países pequeños
    
    print("=== CLASIFICACIÓN POR TIERS ===")
    print(f"Tier 1 (>=1000 trans): {len(tier1)} países - {tier1.sum():,} transacciones")
    print(f"Tier 2 (100-999 trans): {len(tier2)} países - {tier2.sum():,} transacciones")
    print(f"Tier 3 (<100 trans): {len(tier3)} países - {tier3.sum():,} transacciones")
    
    return {
        'tier1': tier1.index.tolist(),
        'tier2': tier2.index.tolist(), 
        'tier3': tier3.index.tolist()
    }

# Agregar columna de tier al dataset
def assign_tier(country):
    if country in tiers['tier1']:
        return 'Tier1_Principal'
    elif country in tiers['tier2']:
        return 'Tier2_Mediano'
    else:
        return 'Tier3_Pequeño'

tiers = classify_countries_by_volume(df)

df['Country_Tier'] = df['Country'].apply(assign_tier)
print("\nDistribución por tiers:")
df['Country_Tier'].value_counts()

In [ ]:
def consolidate_countries(df, umbral_minimo=100):
    country_counts = df['Country'].value_counts()
    
    # Países principales (por encima del umbral)
    main_countries = country_counts[country_counts >= umbral_minimo].index.tolist()
    
    # Crear nueva columna
    df['Country_Consolidated'] = df['Country'].apply(
        lambda x: x if x in main_countries else 'Other_Countries'
    )
    
    print(f"=== CONSOLIDACIÓN DE PAÍSES (umbral: {umbral_minimo}) ===")
    print(f"Países principales: {len(main_countries)}")
    print(f"Países agrupados en 'Other': {len(country_counts) - len(main_countries)}")
    
    consolidation_stats = df['Country_Consolidated'].value_counts()
    print(f"\nDistribución consolidada:")
    for country, count in consolidation_stats.items():
        percent = (count / len(df)) * 100
        print(f"  {country}: {count:,} ({percent:.1f}%)")
    
    return df

df = consolidate_countries(df, umbral_minimo=100)

In [ ]:
# Agrupar por regiones geográficas
def make_regions(country):
    """Mapear países a regiones"""
    europe = ['United Kingdom', 'Germany', 'France', 'Spain', 'Netherlands', 
              'Belgium', 'Switzerland', 'Austria', 'Italy', 'Portugal', 'Norway',
              'Denmark', 'Finland', 'Sweden', 'Poland', 'Cyprus']
    
    asia_pacific = ['Australia', 'Japan', 'Singapore', 'Hong Kong']
    
    americas = ['USA', 'Canada', 'Brazil']
    
    if country in europe:
        return 'Europa'
    elif country in asia_pacific:
        return 'Asia_Pacífico'
    elif country in americas:
        return 'Américas'
    else:
        return 'Otros'

df['Region'] = df['Country'].apply(make_regions)

print("=== DISTRIBUCIÓN POR REGIÓN ===")
region_stats = df['Region'].value_counts()
for region, count in region_stats.items():
    percent = (count / len(df)) * 100
    print(f"{region}: {count:,} ({percent:.1f}%)")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Top 10 países
top_countries = df['Country'].value_counts().head(10)
axes[0,0].bar(range(len(top_countries)), top_countries.values)
axes[0,0].set_title('Top 10 Países por Transacciones')
axes[0,0].set_xticks(range(len(top_countries)))
axes[0,0].set_xticklabels(top_countries.index, rotation=45)

# 2. Distribución por tiers
tier_counts = df['Country_Tier'].value_counts()
axes[0,1].pie(tier_counts.values, labels=tier_counts.index, autopct='%1.1f%%')
axes[0,1].set_title('Distribución por Tiers de País')

# 3. Distribución por región
region_counts = df['Region'].value_counts()
axes[1,0].pie(region_counts.values, labels=region_counts.index, autopct='%1.1f%%')
axes[1,0].set_title('Distribución por Región')

# 4. Países consolidados
consol_counts = df['Country_Consolidated'].value_counts()
axes[1,1].bar(range(len(consol_counts)), consol_counts.values)
axes[1,1].set_title('Países Consolidados')
axes[1,1].set_xticks(range(len(consol_counts)))
axes[1,1].set_xticklabels(consol_counts.index, rotation=45)

plt.tight_layout()
plt.show()

Obteniendo estos datos se obta por un filtrado conbinado para que el conjunto de datos este lo más balanceado posible.

In [ ]:
df_filter = df[
    (df['Country_Tier'].isin(['Tier1_Principal', 'Tier2_Mediano'])) &
    (df['Region'].isin(['Europa', 'Asia_Pacífico', 'Américas']))
]

In [ ]:
print("=== FILTRADO COMBINADO ===")
print(f"Dataset original: {df.shape[0]} filas")
print(f"Dataset filtrado (Tier 1+2 + Regiones principales): {df_filter.shape[0]} filas")
print(f"Filas eliminadas: {df.shape[0] - df_filter.shape[0]}")
print(f"Porcentaje mantenido: {(df_filter.shape[0] / df.shape[0]) * 100:.1f}%")

print("\nDistribución final por tiers:")
print(df_filter['Country_Tier'].value_counts())

print("\nDistribución final por regiones:")
print(df_filter['Region'].value_counts())

print(f"\nClientes únicos después del filtrado: {df_filter['CustomerID'].nunique()}")

In [ ]:
print("=== VALIDACIÓN DEL FILTRADO ===")

# Verificar balance después del filtrado
print("Distribución por tiers (más balanceada):")
tier_dist = df['Country_Tier'].value_counts(normalize=True) * 100
for tier, pct in tier_dist.items():
    print(f"  {tier}: {pct:.1f}%")

print("\nDistribución por regiones (más balanceada):")
region_dist = df['Region'].value_counts(normalize=True) * 100
for region, pct in region_dist.items():
    print(f"  {region}: {pct:.1f}%")

# Verificar que no perdimos demasiados clientes
clients_by_country = df.groupby('Country')['CustomerID'].nunique().sort_values(ascending=False)
print(f"\nTop 5 países por clientes únicos:")
clients_by_country.head()

## 4. Análisis de los Datos sesgados

Se trabajara con la columna UnitPrice, ya que es la que tiene un sego muy alto (γ1 = 186.50).

<img src="../reports/imgs/unitprice.png" alt="Datos imbalanceados">

In [ ]:
print("Estadísticas descriptivas de UnitPrice:")
print(df['UnitPrice'].describe())

In [ ]:
bias = stats.skew(df['UnitPrice'])
kurtosis = stats.kurtosis(df['UnitPrice'])

print(f"\nMedidas de forma:")
print(f"Sesgo (skewness): {bias:.2f}")
print(f"Curtosis (kurtosis): {kurtosis:.2f}")

# Identificar outliers
Q1 = df['UnitPrice'].quantile(0.25)
Q3 = df['UnitPrice'].quantile(0.75)
IQR = Q3 - Q1
lower_limit = Q1 - 1.5 * IQR
upper_limit = Q3 + 1.5 * IQR

outliers = df[(df['UnitPrice'] < lower_limit) | (df['UnitPrice'] > upper_limit)]
print(f"\nOutliers detectados: {len(outliers)} ({len(outliers)/len(df)*100:.2f}%)")
print(f"Rango normal: {lower_limit:.2f} - {upper_limit:.2f}")
print(f"Valor máximo: {df['UnitPrice'].max():.2f}")
print(f"Valor mínimo: {df['UnitPrice'].min():.2f}")


In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(18, 10))

# 1. Distribución original
axes[0,0].hist(df['UnitPrice'], bins=50, alpha=0.7, color='blue')
axes[0,0].set_title('Distribución Original de UnitPrice')
axes[0,0].set_xlabel('UnitPrice')
axes[0,0].set_ylabel('Frecuencia')

# 2. Boxplot original
axes[0,1].boxplot(df['UnitPrice'])
axes[0,1].set_title('Boxplot Original')
axes[0,1].set_ylabel('UnitPrice')

# 3. QQ Plot original
stats.probplot(df['UnitPrice'], dist="norm", plot=axes[0,2])
axes[0,2].set_title('Q-Q Plot Original')

# 4. Distribución sin outliers extremos
unitprice_clean = df[df['UnitPrice'] <= df['UnitPrice'].quantile(0.95)]['UnitPrice']
axes[1,0].hist(unitprice_clean, bins=50, alpha=0.7, color='green')
axes[1,0].set_title('Distribución sin Outliers Extremos (95%)')
axes[1,0].set_xlabel('UnitPrice')
axes[1,0].set_ylabel('Frecuencia')

# 5. Transformación logarítmica
unitprice_log = np.log1p(df[df['UnitPrice'] > 0]['UnitPrice'])
axes[1,1].hist(unitprice_log, bins=50, alpha=0.7, color='red')
axes[1,1].set_title('Distribución Log-transformada')
axes[1,1].set_xlabel('Log(UnitPrice + 1)')
axes[1,1].set_ylabel('Frecuencia')

# 6. Transformación de raíz cuadrada
unitprice_sqrt = np.sqrt(df[df['UnitPrice'] >= 0]['UnitPrice'])
axes[1,2].hist(unitprice_sqrt, bins=50, alpha=0.7, color='orange')
axes[1,2].set_title('Distribución Raíz Cuadrada')
axes[1,2].set_xlabel('√UnitPrice')
axes[1,2].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()


In [ ]:
df_final_filtering = df[
    (df['Country_Tier'].isin(['Tier1_Principal', 'Tier2_Mediano'])) &
    (df['Region'].isin(['Europa', 'Asia_Pacífico', 'Américas']))
].copy()

print("=== FILTRADO COMBINADO ===")
print(f"Dataset original: {df.shape[0]} filas")
print(f"Dataset filtrado (Tier 1+2 + Regiones principales): {df_final_filtering.shape[0]} filas")
print(f"Filas eliminadas: {df.shape[0] - df_final_filtering.shape[0]}")
print(f"Porcentaje mantenido: {(df_final_filtering.shape[0] / df.shape[0]) * 100:.1f}%")

print("\nDistribución final por tiers:")
print(df_final_filtering['Country_Tier'].value_counts())

print("\nDistribución final por regiones:")
print(df_final_filtering['Region'].value_counts())

print(f"\nClientes únicos después del filtrado: {df_final_filtering['CustomerID'].nunique()}")

In [ ]:
def final_strategy(df):
    """Aplicar estrategia final para datos sesgados"""
    
    print("=== ESTRATEGIA FINAL PARA DATOS SESGADOS ===")
    
    # 1. Crear transformación logarítmica para modelado
    df['UnitPrice_Log'] = np.log1p(df['UnitPrice'])
    
    # 2. Aplicar winsorización para outliers extremos
    p99 = df['UnitPrice'].quantile(0.99)
    p1 = df['UnitPrice'].quantile(0.01)
    df['UnitPrice_Clean'] = df['UnitPrice'].clip(lower=p1, upper=p99)
    
    # 3. Crear segmentos de precios para análisis categórico
    # (ya se aplicó arriba)
    
    # 4. Crear variable de valor total
    df['TotalValue'] = df['Quantity'] * df['UnitPrice']
    df['TotalValue_Log'] = np.log1p(df['TotalValue'].clip(lower=0))
    
    print("Variables creadas:")
    print("- UnitPrice_Log: Para modelado ML")
    print("- UnitPrice_Clean: Outliers controlados")
    print("- Price_Segment: Para análisis categórico")
    print("- TotalValue_Log: Valor total transformado")
    
    # Verificar mejora en sesgo
    bias_original = stats.skew(df['UnitPrice'])
    bias_log = stats.skew(df['UnitPrice_Log'])
    bias_clean = stats.skew(df['UnitPrice_Clean'])
    
    print(f"\n=== MEJORA EN SESGO ===")
    print(f"Original: {bias_original:.3f}")
    print(f"Log-transformado: {bias_log:.3f}")
    print(f"Winsorizado: {bias_clean:.3f}")
    
    return df

# Aplicar estrategia final
df = final_strategy(df_final_filtering)

In [ ]:
# Estadísticas finales
print(f"\nDataset final:")
print(f"Filas: {df.shape[0]}")
print(f"Columnas: {df.shape[1]}")

# Verificar que no hay valores nulos en las transformaciones
print(f"\nValores nulos en transformaciones:")
for col in ['UnitPrice_Log', 'UnitPrice_Clean', 'TotalValue_Log']:
    if col in df.columns:
        nulls = df[col].isnull().sum()
        print(f"{col}: {nulls} nulos")

In [ ]:
df

In [ ]:
df.to_csv("../data/data_wrangling.csv")